In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat # 导入matlab文件
from sklearn.preprocessing import PolynomialFeatures

In [3]:
from pylab import mpl

mpl.rcParams['font.sans-serif'] = ['SimHei']
mpl.rcParams['axes.unicode_minus'] = False

In [4]:
def h(theta, x):
    """
    预测函数
    Args:
        theta 模型参数
        x 特征向量
        
    Returns:
        预测结果
    """
    return (theta.T * x)[0,0] # 返回二维array的值

In [ ]:
def J(theta, X, y, theLambda=0):
    """代价函数
    
    Args:
        theta 模型参数
        X 样本特征
        y
    """
    return (X * theta - y).T * (X * theta -y) / (2 * m) + theLambda * np.sum(np.square(theta)) / (2 * m)

In [5]:
def gradient(X, y, alpha = 1, maxLoop = 50, epsilon = 1e-5, theLambda = 0, initTheta = None):
    """批量梯度下降法
    
    Args:
        X 样本特征
        y 样本特征
        alpha 学习率
        epsilon 收敛精度
        theLambda 正则化参数
    
    Returns:
        theta, errors
    """
    m, n = X.shape
    if initTheta is None:
        theta = np.zeros((n, 1))
    else:
        theta = initTheta
    count = 0
    
    error = float('inf')
    errors = [errors, ]
    for i in range(maxLoop):
        theta = theta + (1.0 / m) * alpha * ((y - X * theta).T * X).T
        error = J(theta, X, y, theLambda)
        if np.isnan(error):
            error = np.inf
        
        errors.append(error)
        if(abs(errors[-1]-errors[-2]) < epsilon):
            break
    print('iterating', i)
    return theta, errors
            

In [6]:
def normalize(X):
    """特征归一化处理
    主要目的是加快迭代速度，有多个特征并且量级相差较大的时候使用
    
    Args:
        X 样本集
    
    Returns:
        归一化后的样本集
    """
    
    m, n = X.shape
    # 归一化每一个特征
    for j in range(n):
        features = X[:, j]
        minVal = features.min(axis = 0)
        maxVal = features.max(axis = 0)
        diff = maxVal - minVal
        if diff != 0:
            X[:, j] = (features -  minVal) / diff
        else:
            X[:, j] = 0
    return X

In [7]:
filename = 'data/water.mat'

In [ ]:
data = loadmat(filename)
####
# 数据集划分
####
# 训练集合
X = np.mat(data['X'])
X = np.concatenate((np.ones(X.shape[0],1), X), axis = 1) # 添加偏置
y = np.mat(data['y'])
# 交叉验证集
Xval = np.mat(data['Xval'])
Xval = np.concatenate((np.ones(Xval.shape[0], 1), Xval), axis = 1)
yval = np.mat(data['yval'])
# 测试集
Xtest = np.mat(data['Xtest'])
Xtest = np.concatenate((np.ones(Xtest.shape[0], 1), Xtest), axis = 1)
ytest = np.mat(data['ytest'])

In [ ]:
def diagnoseLR():
    """线性回归诊断
    
    """
    initTheta = np.mat(np.ones((X.shape[1], 1)))
    theta, errors = gradient(
        X, y , alpha = 0.001, maxLoop = 5000, epsilon = 0.00001, initTheta = initTheta
    )
    
    # 绘制拟合成果
    Xmin = X[:, 1].min()
    Xmax = X[:, 1].max()
    ymax = y[:, 0].max()
    ymin = y[:, 0].min()
    fixX = np.mat(np.linspace(Xmin, Xmax, 20).reshape(-1, 1))
    fixX = np.concatenate((np.ones((fitX.shape[0], 1)), fixX), axis = 1)
    
    h = fixX * theta
    plt.xlim(Xmin, Xmax)
    plt.ylim(ymin, ymax)
    # 绘制训练样本
    plt.scatter(X[:, 1].flatten().A[0], y[:, 0].flatten().A[0], marker = 'x', color = 'r', linewidth = 2)
    # 绘制拟合曲线
    plt.plot(fitX[:, 1], h, color = 'b')
    plt.xlabel(u'水位变化(x)')
    plt.ylabel(u'大坝流量(y)')
    plt.show()
    
    # 绘制随样本规模学习曲线
    m, n = X.shape
    trainErrors = np.zeros((1, m))
    valErrors = np.zeros((1, m))
    for i in range(m):
        Xtrain = X[0:i+1]
        ytrain = y[0:i+1]
        # 注意，这里我们没有设置theLambda，并且这里特征只使用了一个
        theta, errors = gradient(
            Xtrain, ytrain, alpha = 0.001, maxLoop = 10000, epsilon = 0.00001
        )
        trainErrors[0, i] = J(theta, Xtrain, ytrain)
        valErrors[0, i] = J(theta, Xval, yval)
    
    print(u'最小交叉验证误差', valErrors.ravel()[-1])
    plt.plot(np.arange(1, m+1).ravel(), trainErrors.ravel(), color = 'b', label = u'测试误差')
    plt.plot(np.arange(1, m+1).ravel(), valErrors.ravel(), color = 'g', label = u'交叉验证误差')
    plt.title(u'线性回归学习曲线')
    plt.xlabel(u'训练样本集')
    plt.ylabel(u'误差')
    plt.legend()
    plt.show()

In [8]:
def diagnosePR():
    """多项式回归诊断
    
    """
    poly = PolynomialFeatures(degree = 10)
    # 注意使用normalize做了标准化
    XX, XXval, XXtest = 
        [normalize(np.mat(ploy.fit_transform(data[:, 1:]))) for data in [X, Xval, Xtest]]
    initTheta = np.mat(np.ones((XX.shape[1], 1)))
    theLambdas = [0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.10, 0.11,
                 0.12, 0.13, 0.14, 0.15, 0.2, 0.3, 0.5]
    numTheLambdas = len(theLambdas)
    trainErrors = np.zeros((1, numTheLambdas))
    valErrors = np.zeros((1, numTheLambdas))
    thetas = []
    for idx, theLambda in enumerate(theLambdas):
        theta, errors = gradient(
            XX, y, alpha = 0.1, maxLoop = 10000, epsilon = 0.0001，
            theLambda = theLambda, initTheta = initTheta
        )
        thetas.append(theta)
        # 训练误差、交叉验证误差，不需要考虑模型复杂度， 所以theLambda不需要设置或着设置成0
        trainErrors[0, idx] = J(theta, XX, y)
        valErrors[0, idx] = J(theta, XXval, yval)
        print(theLambda, valErrors[0, idx])
    
    bestLambda = theLambdas[np.argmin(valErrors)]
    theta = thetas[np.argmin(valErrors)]
    error = np.min(valErrors)
    
    plt.plot(theLambdas, trainErrors.ravel(), color = 'b', label = u'训练误差')
    plt.plot(theLambdas, valErrors.ravel(), color = 'g', label = u'交叉验证误差')
    plt.title(u'多项式回归线则lambda')
    plt.xlabel(u'lambda')
    plt.ylabel(u'误差')
    plt.legend()
    plt.show()
    
    # 绘制拟合曲线
    fitX = np.mat(np.linspace(-60, 45).reshape(-1, 1))
    fitX = np.concatenate((np.ones((fitX.shape[0], 1)), fitX), axis = 1)
    fitXX = normalize(np.mat(poly.fit_transform(fitX[:, 1:])))
    h = fitXX * theta
    print(theta)
    plt.title(u'多项式回归拟合曲线(lambda = %.3f) \n 交叉验证误差=%.3f' % (bestLambda, error))
    
    plt.scatter(X[:, 1].tolist(), y[:, 0].tolist(), marker = 'x', color = 'r', linewidth = 3)
    plt.plot(fitX[:, 1], h, color = 'b')
    plt.show()
        
    

SyntaxError: unexpected EOF while parsing (<ipython-input-8-6645a3766ff6>, line 1)